<div style="background-color:#000;"><img src="pqn.png"></img></div>

These libraries enable us to work with advanced financial modeling and asynchronous programming

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import (
    FunctionAgent,
    AgentWorkflow,
    AgentOutput,
    ToolCall,
    ToolCallResult,
)
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex

We start by loading environment variables and initializing our language model

In [ ]:
load_dotenv()

In [ ]:
llm = OpenAI(model="gpt-4o")

This code sets up our environment and initializes the language model. We load environment variables, which likely include API keys. Then we create an instance of the OpenAI language model, specifically using the GPT-4 model. This prepares us for natural language processing tasks.

## Define our tool functions

We define asynchronous functions that will serve as tools for our agents

In [ ]:
async def read_pdf_tool(ctx: Context) -> str:
    documents = SimpleDirectoryReader(input_files=["pairs.pdf"]).load_data()
    index = GPTVectorStoreIndex(documents)
    query_engine = index.as_query_engine()
    query = (
        "Extract a detailed description of the pairs trading strategy implementation from this PDF. "
        "Ensure the description is detailed enough to reproduce the strategy in code."
    )
    response = query_engine.query(query)
    return str(response)

In [ ]:
async def build_plan_tool(ctx: Context, plan: str) -> str:
    current_state = await ctx.get("state")
    current_state["implementation_plan"] = plan
    await ctx.set("state", current_state)
    return "Implementation plan recorded."

In [ ]:
async def write_code_tool(ctx: Context, plan: str) -> str:
    current_state = await ctx.get("state")
    current_state["python_code"] = plan
    await ctx.set("state", current_state)
    return "Python code recorded."

These functions are tools that our agents will use. The read_pdf_tool extracts information from a PDF file about a pairs trading strategy. The build_plan_tool and write_code_tool update the workflow state with an implementation plan and Python code, respectively. These tools allow our agents to process information and generate outputs.

## Create our function agents

We define three function agents to handle different tasks in our workflow

In [ ]:
pdf_reader_agent = FunctionAgent(
    name="PDFReaderAgent",
    description=(
        "Reads a PDF file containing a pairs trading strategy and extracts a detailed description "
        "of the strategy implementation."
    ),
    system_prompt=(
        "You are the PDFReaderAgent that can read PDFs containing implementation details of pairs trading strategies "
        "and describe the strategy in detail. Once you read the PDF and describe the implementation details and are "
        "satisfied, you should hand off control to the PlanBuilderAgent to develop an implementation plan. "
        "You should have a detailed description of the strategy before handing off control to the PlanBuilderAgent."
    ),
    llm=llm,
    tools=[read_pdf_tool],
    can_handoff_to=["PlanBuilderAgent"],
)

In [ ]:
plan_builder_agent = FunctionAgent(
    name="PlanBuilderAgent",
    description=(
        "Takes the detailed strategy description and builds a detailed plan to implement the strategy in Python."
    ),
    system_prompt=(
        "You are the PlanBuilderAgent. Your task is to analyze the strategy description from the state and generate "
        "a detailed plan outlining the steps, functions, and code structure required to implement the pairs trading strategy. "
        "Include suggested Python libraries for the implementation. Your plan should be in markdown format. Once the plan "
        "is written, you should hand off control to the CodeWriterAgent. Output your plan with no preamble. Just output the plan."
    ),
    llm=llm,
    tools=[build_plan_tool],
    can_handoff_to=["CodeWriterAgent"],
)

In [ ]:
code_writer_agent = FunctionAgent(
    name="CodeWriterAgent",
    description=(
        "Takes the implementation plan and writes complete Python code for the pairs trading strategy."
    ),
    system_prompt=(
        "You are the CodeWriterAgent. Your task is to use the implementation plan from the state to write complete "
        "and executable Python code for the pairs trading strategy. Output your Python code with no preamble. Just "
        "output the Python code."
    ),
    llm=llm,
    tools=[write_code_tool],
    can_handoff_to=[],
)

We create three function agents: PDFReaderAgent, PlanBuilderAgent, and CodeWriterAgent. Each agent has a specific role in our workflow. They use the language model and their assigned tools to perform tasks like reading PDFs, building implementation plans, and writing code. These agents work together to process information and generate outputs.

## Set up our agent workflow

We create an agent workflow to orchestrate the interaction between our agents

In [ ]:
agent_workflow = AgentWorkflow(
    agents=[pdf_reader_agent, plan_builder_agent, code_writer_agent],
    root_agent=pdf_reader_agent.name,
    initial_state={
        "strategy_description": "Not generated yet.",
        "implementation_plan": "Not generated yet.",
        "python_code": "Not generated yet.",
    },
)

This code sets up our agent workflow. It defines the sequence of agents that will work on our task, starting with the PDFReaderAgent. We also set an initial state for our workflow, which will be updated as the agents perform their tasks. This workflow structure allows our agents to collaborate effectively.

## Execute our workflow

We run our agent workflow and process its output

In [ ]:
user_msg = (
    "Please process the PDF file 'pairs.pdf'. "
    "Extract a detailed description of the pairs trading strategy, build an implementation plan in Python, "
    "and finally generate the complete Python code for the strategy."
)

In [ ]:
handler = agent_workflow.run(user_msg=user_msg)
current_agent = None
async for event in handler.stream_events():
    if hasattr(event, "current_agent_name") and event.current_agent_name != current_agent:
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print("🛠️  Planning to use tools:", [call.tool_name for call in event.tool_calls])
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print("  Arguments:", event.tool_kwargs)
        print("  Output:", event.tool_output)
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print("  With arguments:", event.tool_kwargs)

We execute our agent workflow by providing a user message that outlines the task. The code then processes the events streamed by the workflow. It prints information about which agent is currently active, their outputs, tool usage, and results. This allows us to see the step-by-step progress of our workflow as it processes the PDF, builds a plan, and generates code for the pairs trading strategy.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.